# Tutorial: Homomorphisms

This is an interactive tutorial written with real code.
We start by setting up $\LaTeX$ printing.

In [1]:
from IPython.display import display, Math

def show(arg):
    return display(Math(arg.to_latex()))

## Initializing a homomorphism

Homomorphisms between general LCAs are represented by the `HomLCA` class.
To define a homomorphism, a matrix representation is needed.
In addition to the matrix, the user can also define a `target` and `source`. 

Some verification of the inputs is done in the initializer, for instance a matrix $A \in \mathbb{Z}^{2 \times 2}$ cannot represent $\phi: \mathbb{Z}^m \to \mathbb{Z}^n$ unless both $m$ and $n$ are $2$.
If no `target`/`source` is given, the initializer
will assume a free, discrete group, i.e. $\mathbb{Z}^m$.

In [2]:
from abelian import LCA, HomLCA

# Initialize the target group for the homomorphism
target = LCA([0, 5], discrete = [False, True])

# Initialize a homomorphism between LCAs
phi = HomLCA([[1, 2], [3, 4]], target = target)
show(phi)

# Initialize a homomorphism with no source/target.
# Souce and targets are assumed to be
# of infinite order and discrete (free-to-free)
phi = HomLCA([[1, 2], [3, 4]])
show(phi)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Homomorphisms between FGAs are represented by the `HomFGA` class.

In [3]:
from abelian import HomFGA
phi = HomFGA([[4, 5], [9, -3]])
show(phi)

<IPython.core.display.Math object>

The `HomFGA` class is a subclass of `HomLCA`. 
A `HomFGA` instance must have:

* FGAs as source and target
* The matrix must contain only integer entries

In [4]:
issubclass(HomFGA, HomLCA) # Verify subclass

True

The best way to create a `HomLCA` or `HomFGA` is to use the `Homomorphism` factory function. 
It will attempt to return an instance of the correct class depending on the data passed to it.

In [5]:
from abelian import Homomorphism

# Integer matrix -> returns HomFGA
A = [[1, 0, 1], [0, 1, 1]]
psi = Homomorphism(A)
assert type(psi) == HomFGA

# Non-integer matrix -> returns HomLCA
R2 = LCA([0, 0], [False, False])
psi = Homomorphism([[1, 0, 2.3], [0, 1, 1]], R2)
assert type(psi) == HomLCA

# Non-FGA target -> returns HomLCA
A = [[1, 0, 1], [0, 1, 1]]
G = LCA([1, 1], [False, False])
psi = Homomorphism(A, target = G)
assert type(psi) == HomLCA

## Compositions

A fundamental way to combine two functions is to compose them.
We create two homomorphisms and compose them: first $\psi$, then $\phi$.
The result is the function $\phi \circ \psi$.

In [6]:
# Create two HomFGAs
phi = HomFGA([[4, 5], [9, -3]])
psi = Homomorphism([[1, 0, 1], [0, 1, 1]])

# The composition of phi, then psi
show(phi * psi)

<IPython.core.display.Math object>

If the homomorphism is an endomorphism (same source and target),
repeated composition can be done using exponents.

$\phi^{n} = \phi \circ \phi \circ \dots \circ \phi, \quad n \geq 1$

In [7]:
show(phi**3)

<IPython.core.display.Math object>

Numbers and homomorphisms can be added to homomorphisms, 
in the same way that numbers and matrices are added to matrices in other software.

In [8]:
show(psi)

# Each element in the matrix is multiplied by 2
show(psi + psi)

# Elementwise addition
show(psi + 10)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Slice notation

Slice notation is available. The first slice works on rows (target group)
and the second slice works on columns (source group).
Notice that in Python, indices start with 0.

In [9]:
A = [[10, 10], [10, 15]]
# Notice how the Homomorphism converts a list
# into a FGA, this makes it easier to create HomFGAs
phi = Homomorphism(A, target = [20, 20])
phi = phi.project_to_source()

# Slice in different ways
show(phi)
show(phi[0, :]) # First row, all columns
show(phi[:, 0]) # All rows, first column
show(phi[1, 1]) # Second row, second column

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Stacking homomorphisms

There are three ways to stack morphisms:

* Diagonal stacking
* Horizontal stacking
* Vertical stacking

They are all shown below.

### Diagonal stacking

In [10]:
# Create two homomorphisms
phi = Homomorphism([2], target = LCA([0], [False]))
psi = Homomorphism([2])

# Stack diagonally
show(phi.stack_diag(psi))

<IPython.core.display.Math object>

### Horizontal stacking

In [11]:
# Create two homomorphisms with the same target
target = LCA([0], [False])
phi = Homomorphism([[1, 3]], target = target)
source = LCA([0], [False])
psi = Homomorphism([7], target=target, source=source)

# Stack horizontally
show(phi.stack_horiz(psi))

<IPython.core.display.Math object>

### Vertical stacking

In [12]:
# Create two homomorphisms, they have the same source
phi = Homomorphism([[1, 2]])
psi = Homomorphism([[3, 4]])

# Stack vertically
show(phi.stack_vert(psi))

<IPython.core.display.Math object>

## Calling homomorphisms

In Python, a `callable` is an object which implements a method for function calls.
A homomorphism is a callable object, so we can use `phi(x)` to evaluate `x`, i.e. send `x` from the source to the target.

We create a homomorphism.

In [13]:
# Create a homomorphism, specify the target
phi = Homomorphism([[2, 0], [0, 4]], [10, 12])
# Find the source group (orders)
phi = phi.project_to_source()
show(phi)

<IPython.core.display.Math object>

We can now call it. The argument must be in the source group.

In [14]:
# An element in the source, represented as a list
group_element = [1, 1]
# Calling the homomorphism
print(phi(group_element))

# Since [6, 4] = [1, 1] mod [5, 3] (source group)
# the following is equal
print(phi([6, 4]) == phi([1, 1]))

[2, 4]
True


## Calling and composing

We finish this tutorial by showing two ways to calculate the same thing:

* $y = (\phi \circ \psi)(x)$
* $y = \phi(\psi(x))$

In [15]:
# Create two HomFGAs
phi = HomFGA([[4, 5], [9, -3]])
psi = Homomorphism([[1, 0, 1], [0, 1, 1]])

x = [1, 1, 1]
# Compose, then call
answer1 = (phi * psi)(x)

# Call, then call again
answer2 = phi(psi(x))

# The result is the same
print(answer1 == answer2)

True
